# Qdrant Quickstart from [docs](https://qdrant.tech/documentation/quick-start/) 

> Run from terminal

In [ ]:
#docker pull qdrant/qdrant

In [1]:
#docker run -p 6333:6333 --name qdrant -v $(pwd)/qdrant_storage:/qdrant/storage qdrant/qdrant

In [2]:
!docker logs qdrant --tail 20

           _                 _    
  __ _  __| |_ __ __ _ _ __ | |_  
 / _` |/ _` | '__/ _` | '_ \| __| 
| (_| | (_| | | | (_| | | | | |_  
 \__, |\__,_|_|  \__,_|_| |_|\__| 
    |_|                           

Access web UI at http://localhost:6333/dashboard

[2023-08-18T11:50:05.496Z INFO  storage::content_manager::consensus::persistent] Loading raft state from ./storage/raft_state.json
[2023-08-18T11:50:05.548Z INFO  storage::content_manager::toc] Loading collection: first collection
[2023-08-18T11:50:05.843Z INFO  qdrant] Distributed mode disabled
[2023-08-18T11:50:05.843Z INFO  qdrant] Telemetry reporting enabled, id: d4768c48-411b-4786-aafe-a34153bb3f8a
[2023-08-18T11:50:05.843Z INFO  qdrant::tonic] Qdrant gRPC listening on 6334
[2023-08-18T11:50:05.843Z INFO  qdrant::tonic] TLS disabled for gRPC API
[2023-08-18T11:50:05.844Z INFO  qdrant::actix] TLS disabled for REST API
[2023-08-18T11:50:05.844Z INFO  qdrant::actix] Qdrant HTTP listening on 6333
[2023-08-18T11:50:05.844Z INFO  

In [3]:
pip install qdrant-client

Note: you may need to restart the kernel to use updated packages.


In [4]:
from qdrant_client import QdrantClient
client = QdrantClient(host="localhost", port=6333)
client

## Create collection

In [5]:
from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

client = QdrantClient("localhost", port=6333)
client.recreate_collection(
    collection_name="test_collection",
    vectors_config=VectorParams(size=4, distance=Distance.DOT),
)

True

In [6]:
collection_info = client.get_collection(collection_name="test_collection")

In [7]:
from qdrant_client.http.models import CollectionStatus

assert collection_info.status == CollectionStatus.GREEN
assert collection_info.vectors_count == 0

## Add points

In [8]:
from qdrant_client.http.models import PointStruct

operation_info = client.upsert(
    collection_name="test_collection",
    wait=True,
    points=[
        PointStruct(id=1, vector=[0.05, 0.61, 0.76, 0.74], payload={"city": "Berlin"}),
        PointStruct(id=2, vector=[0.19, 0.81, 0.75, 0.11], payload={"city": ["Berlin", "London"]}),
        PointStruct(id=3, vector=[0.36, 0.55, 0.47, 0.94], payload={"city": ["Berlin", "Moscow"]}),
        PointStruct(id=4, vector=[0.18, 0.01, 0.85, 0.80], payload={"city": ["London", "Moscow"]}),
        PointStruct(id=5, vector=[0.24, 0.18, 0.22, 0.44], payload={"count": [0]}),
        PointStruct(id=6, vector=[0.35, 0.08, 0.11, 0.44]),
    ]
)

In [9]:
from qdrant_client.http.models import UpdateStatus

assert operation_info.status == UpdateStatus.COMPLETED

## Search with filtering

In [10]:
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7], 
    limit=3
)

In [11]:
assert len(search_result) == 3

print(search_result[0])
# ScoredPoint(id=4, score=1.362, ...)

print(search_result[1])
# ScoredPoint(id=1, score=1.273, ...)

print(search_result[2])
# ScoredPoint(id=3, score=1.208, ...)

id=4 version=0 score=1.362 payload={'city': ['London', 'Moscow']} vector=None
id=1 version=0 score=1.273 payload={'city': 'Berlin'} vector=None
id=3 version=0 score=1.208 payload={'city': ['Berlin', 'Moscow']} vector=None


> Show different search result when filter added

In [12]:
from qdrant_client.http.models import Filter, FieldCondition, MatchValue
search_result = client.search(
    collection_name="test_collection",
    query_vector=[0.2, 0.1, 0.9, 0.7], 
    query_filter=Filter(
        must=[
            FieldCondition(
                key="city",
                match=MatchValue(value="London")
            )
        ]
    ),
    limit=3
)

In [13]:
assert len(search_result) == 2

print(search_result[0])
# ScoredPoint(id=4, score=1.362, ...)

print(search_result[1])
# ScoredPoint(id=2, score=0.871, ...)

id=4 version=0 score=1.362 payload={'city': ['London', 'Moscow']} vector=None
id=2 version=0 score=0.871 payload={'city': ['Berlin', 'London']} vector=None
